### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [ ]:
import numpy as np
import pandas as pd
import json
import codecs
import os
import sys
sys.path.append('../')
import oci
from oci.config import validate_config
import ocifs
import ads

In [ ]:
bucket_name = "tenen-raw-data"
config = oci.config.from_file(
        "~/.apikey/config",
        "DEFAULT")

In [ ]:
object_storage_client = oci.object_storage.ObjectStorageClient(config)
namespace = object_storage_client.get_namespace().data
namespace

In [ ]:
def write2df(path):
    df = pd.read_csv(path, low_memory=False) # "/home/datascience/files/tenen_highlvl_oct.csv"
    return df    

In [ ]:
target_file = {'oct' : 'tenen_high_lvl_oct.csv', 'nov': 'tenen_high_lvl_nov.csv', 'dec': 'tenen_high_lvl_dec.csv' }
products_file = {'oct' : 'tenen_products_oct.csv', 'nov': 'tenen_products_nov.csv', 'dec': 'tenen_products_dec.csv'}
hits_file = {'oct' : 'tenen_hits_extend_oct.csv', 'nov': 'tenen_hits_extend_nov.csv', 'dec': 'tenen_hits_extend_dec.csv'}

In [ ]:
fname = target_file['oct']
fname

In [ ]:
directory = 'products'
config = {"config": "~/.apikey/config"}
target = f'oci://{bucket_name}@{namespace}/{directory}/{fname}'
target, path

In [ ]:
path = f"/home/datascience/files/{fname}"

In [ ]:
df = write2df(path)

In [ ]:
df.shape

In [ ]:
df.head(5)

In [ ]:
df = pd.read_csv(target, storage_options={"config": "~/.apikey/config"},low_memory=False)

In [ ]:
df.to_csv(target, index=False, encoding='utf-8',storage_options = config)
#put_file(path+"tenen_highlvl_oct.csv", namespace, '/products/tenen_high_lvl_oct.csv')

In [ ]:
from ads.dataset.factory import DatasetFactory
ds = DatasetFactory.open(df) 

In [ ]:
ds.show_in_notebook()